In [4]:
import mga.dataset as dataset
from pathlib import Path
import pandas as pd

In [5]:
dataset.download_ml_small(force_download=True)

'ratings.csv'

In [ ]:
dataset.download_ml_25m(force_download=True)

In [11]:
movie_df = pd.read_csv('movies.csv')
link_df = pd.read_csv('links.csv')

In [12]:
movie_df['imdb_id'] = link_df['imdbId']
movie_df['tmdb_id'] = link_df['tmdbId']
movie_df = movie_df.rename(columns={'movieId': 'movielens_id'})
movie_df.to_csv('movies.csv', index_label='id')

In [13]:
genre_set = set()
for genre_str in movie_df['genres']:
    genre_lst = genre_str.split('|')
    for genre in genre_lst:
        genre_set.add(genre)

no_genre_key = '(no genres listed)'

count_movie_with_no_genre = sum(movie_df['genres'] == no_genre_key)
print(f'Total number of movies with no genre: {count_movie_with_no_genre}')
genre_set.remove(no_genre_key)

ignored_genre_set = {'IMAX', 'Film-Noir', 'Animation', 'Documentary'}

print(f'Genres ({len(genre_set)}): {genre_set}')
print(f'Ignored genres: {ignored_genre_set}')

with open('genre.txt', 'w') as file:
    for genre in genre_set:
        file.write(f'{genre}\n')

Total number of movies with no genre: 34
Genres (19): {'Documentary', 'Animation', 'Film-Noir', 'Children', 'Western', 'Crime', 'IMAX', 'Action', 'Thriller', 'War', 'Mystery', 'Romance', 'Fantasy', 'Sci-Fi', 'Drama', 'Comedy', 'Horror', 'Adventure', 'Musical'}
Ignored genres: {'IMAX', 'Film-Noir', 'Animation', 'Documentary'}


In [14]:
rating_df = pd.read_csv('ratings.csv')
del rating_df['timestamp']

movilens2id = pd.DataFrame(data=movie_df.index, index=movie_df['movielens_id'])
rating_df['movieId'] = movilens2id.loc[rating_df['movieId']].values

id2movielens = pd.Series(rating_df['userId'].drop_duplicates().sort_values().values)
movielens2id = pd.Series(data=id2movielens.index, index=id2movielens)
rating_df['userId'] = movielens2id.loc[rating_df['userId']].values

rating_df = rating_df.rename(columns={'userId': 'user_id'})
rating_df = rating_df.rename(columns={'movieId': 'movie_id'})

In [15]:
row_count = rating_df['user_id'].max() + 1
col_count = rating_df['movie_id'].max() + 1
nnz_count = rating_df.shape[0]

with open('ratings.mtx', 'w') as file:
    file.write(f'{row_count} {col_count} {nnz_count}\n')
    rating_df.to_csv(file, index=False, sep=' ', header=False)

In [16]:
Path("ratings.csv").unlink()
Path("links.csv").unlink()